In [ ]:
!pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# %%capture
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from tqdm import tqdm
import torch
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModel
from transformers import EarlyStoppingCallback

In [ ]:
train_file_name = None
lang = 'Hindi'
train_file_name = pd.read_csv("/content/constraint_Hindi_Train - Sheet1 (1).csv")
dev_file_name = pd.read_csv("/content/Constraint_Hindi_Valid - Sheet1 (1).csv")
test_file_name = pd.read_csv("/content/Test Set Complete - test (1).csv")
label_to_index = {'hate,offensive':0,
                  'non-hostile':1,
                  'defamation,offensive':2,
                  'fake':3,
                  'hate':4,
                  'offensive':5,
                  'fake,hate':6,
                  'defamation':7,
                  'defamation,hate':8,
                  'defamation,hate,offensive':9,
                  'defamation,fake,offensive':10,
                  'fake,offensive':11,
                  'defamation,fake':12,
                  'defamation,fake,hate':13,
                  'fake,hate,offensive':14,
                  'defamation,fake,hate,offensive':15
                  }

class_list = ['hate,offensive',
 'non-hostile',
 'defamation,offensive',
 'fake',
 'hate',
 'offensive',
 'fake,hate',
 'defamation',
 'defamation,hate',
 'defamation,hate,offensive',
 'defamation,fake,offensive',
 'fake,offensive',
 'defamation,fake',
 'defamation,fake,hate',
 'fake,hate,offensive',
 'defamation,fake,hate,offensive']


In [ ]:
class_list

['hate,offensive',
 'non-hostile',
 'defamation,offensive',
 'fake',
 'hate',
 'offensive',
 'fake,hate',
 'defamation',
 'defamation,hate',
 'defamation,hate,offensive',
 'defamation,fake,offensive',
 'fake,offensive',
 'defamation,fake',
 'defamation,fake,hate',
 'fake,hate,offensive',
 'defamation,fake,hate,offensive']

In [ ]:
train_df = pd.read_csv("/content/constraint_Hindi_Train - Sheet1 (1).csv")
train_df.columns = ['Post', 'Label Set']
train_df = train_df.dropna().drop_duplicates().reset_index(drop=True, inplace=False)

val_df = pd.read_csv("/content/Constraint_Hindi_Valid - Sheet1 (1).csv")
val_df.columns = ['Post', 'Label Set']

test_df = pd.read_csv("/content/Test Set Complete - test (1).csv")
test_df.columns = ['Post', 'Label Set']

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased')

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
class MuRILModel(torch.nn.Module):
  def __init__(self, n_classes):
    super(MuRILModel, self).__init__()
    self.l1 = AutoModel.from_pretrained("google/muril-base-cased")
    self.pre_classifier = torch.nn.Linear(768, 768)
    self.dropout = torch.nn.Dropout(0.3)
    self.classifier = torch.nn.Linear(768, n_classes)

  def forward(self, input_ids, attention_mask, token_type_ids):
    output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    pooler = self.pre_classifier(output_1['pooler_output'])
    pooler = torch.nn.ReLU()(pooler)
    pooler = self.dropout(pooler)
    output = self.classifier(pooler)
    return output

class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.sent = dataframe.Post
    self.targets = dataframe['Label Set']
    self.max_len = max_len

  def __len__(self):
    return len(self.sent)

  def __getitem__(self, index):
    sent = str(self.sent[index])
    sent = " ".join(sent.split())

    inputs = self.tokenizer.encode_plus(sent,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        pad_to_max_length=True,
                                        return_token_type_ids=True
                                      )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]


    return {
      'ids': torch.tensor(ids, dtype=torch.long),
      'mask': torch.tensor(mask, dtype=torch.long),
      'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
      'targets': torch.tensor(self.targets[index], dtype=torch.long)
    }

class EarlyStopping():
  """
  Early stopping to stop the training when the loss does not improve after
  certain epochs.
  """
  def __init__(self, patience=5, min_delta=0):
    """
    :param patience: how many epochs to wait before stopping when loss is
            not improving
    :param min_delta: minimum difference between new loss and old loss for
            new loss to be considered as an improvement
    """
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.best_loss = None
    self.early_stop = False

  def __call__(self, val_loss):
    if self.best_loss == None:
      self.best_loss = val_loss
    elif self.best_loss - val_loss > self.min_delta:
      self.best_loss = val_loss
    elif self.best_loss - val_loss < self.min_delta:
      self.counter += 1
      print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
      if self.counter >= self.patience:
        print('INFO: Early stopping')
        self.early_stop = True

In [ ]:
train_df['Label Set'] = train_df['Label Set'].apply(lambda x: label_to_index[x])
val_df['Label Set'] = val_df['Label Set'].apply(lambda x: label_to_index[x])

In [ ]:
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
from torch.utils.data import DataLoader

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

model = MuRILModel(len(class_list))
model = model.float()
model.to(device)
print('Loaded!')

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Loaded!


In [ ]:
# calculate class weights
inverse_weights = np.array(train_df['Label Set'].value_counts().sort_index())
class_weights = np.sum(inverse_weights) / inverse_weights

class_weights = (torch.tensor(class_weights, dtype=torch.float)).cuda()

In [ ]:
loss_function = torch.nn.CrossEntropyLoss(weight = class_weights)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
  n_correct = (preds==targets).sum().item()
  return n_correct

In [ ]:
losslsit = []
acclist = []

In [ ]:
def train(epoch):
  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  model.train()

  for _,data in tqdm(enumerate(training_loader, 0)):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask, token_type_ids)
    loss = loss_function(outputs, targets)
    print(loss)
    tr_loss += loss.item()
    big_val, big_idx = torch.max(outputs.data, dim=1)
    n_correct += calcuate_accuracy(big_idx, targets)
    nb_tr_steps += 1
    nb_tr_examples+=targets.size(0)
    #print(targets)
    #print(big_idx)
    optimizer.zero_grad()
    loss.backward()
    # # When using GPU
    optimizer.step()

  print(f'The Total Accuracy for Epoch {epoch} : {(n_correct*100)/nb_tr_examples}')
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  losslsit.append(float(epoch_loss))
  acclist.append(float(epoch_accu))
  print(f"Training Loss Epoch: {epoch_loss}")
  print(f"Training Accuracy Epoch: {epoch_accu}")
  torch.save(model.state_dict(), f'/content/{lang}_{epoch}.pt', )

In [ ]:
EPOCHS=5

for epoch in range(EPOCHS):
    train(epoch)


print('training Complete...')
print('saving model...')

torch.save(model.state_dict(), f'/content/{lang}_final.pt', )

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:03,  3.46s/it]

tensor(2.7771, device='cuda:0', grad_fn=<NllLossBackward0>)


2it [00:03,  1.69s/it]

tensor(2.7787, device='cuda:0', grad_fn=<NllLossBackward0>)


3it [00:04,  1.14s/it]

...

357it [03:18,  1.80it/s]

tensor(1.3722, device='cuda:0', grad_fn=<NllLossBackward0>)


358it [03:18,  1.80it/s]

tensor(0.3192, device='cuda:0', grad_fn=<NllLossBackward0>)
The Total Accuracy for Epoch 4 : 72.47770589263857
Training Loss Epoch: 2.059949273919926
Training Accuracy Epoch: 72.47770589263857


training Complete...
saving model...


In [ ]:
# # Save the model to the Hugging Face Hub
# model_name = 'your-username/your-model-name'
# model.save_pretrained(model_name)

In [ ]:
PATH = "/content/Hindi_final.pt"
model = MuRILModel(n_classes=16)
model.load_state_dict(torch.load(PATH))
model.eval()

MuRILModel(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = MuRILModel(n_classes=16)  # Replace NUM_CLASSES with your actual number of classes
model.load_state_dict(torch.load(PATH))
model.eval()

# Input text
input_text = "हर तीन महीने में कांग्रेस पार्टी गर्भवती होकर ऑपरेशन थिएटर में भर्ती होती है.. खूब हंगामा मचता है कि इस बार जरूर कुछ होगा जरूर कुछ होगा फिर डॉक्टर ऑपरेशन थिएटर से बाहर निकलता है अपना मास्क निकालता है ग्लब्स उतारता है और कहता है सॉरी बस पेट में गैस था.. रिलीज हो गया....!🤣"

# Tokenize the input text
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=MAX_LEN,  # Define MAX_LEN according to your model's requirements
    padding="max_length",  # Pad to a specific length
    truncation=True,  # Explicitly truncate if the text exceeds MAX_LEN
    return_token_type_ids=True,
    return_tensors="pt"  # Return PyTorch tensors
)

# Extract input IDs, attention mask, and token type IDs
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
token_type_ids = inputs["token_type_ids"]

# Perform inference on the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

# Process the outputs as needed for your task (e.g., softmax for probabilities)
# For example, if you're performing classification, you might do:
softmax_outputs = torch.nn.functional.softmax(outputs, dim=1)
predicted_class = torch.argmax(softmax_outputs, dim=1)

# # Output class probabilities or the predicted class
# print(predicted_class)

# Extract the predicted class value as a Python integer
# predicted_class_value = predicted_class.item()
# print(predicted_class_value)

# Map the value of predicted_class_value to the class_names
predicted_class_name = class_names[predicted_class_value]
print(predicted_class_name)


5


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = MuRILModel(n_classes=16)  # Replace NUM_CLASSES with your actual number of classes
model.load_state_dict(torch.load(PATH))
model.eval()

# Input text
input_text = "हर तीन महीने में कांग्रेस पार्टी गर्भवती होकर ऑपरेशन थिएटर में भर्ती होती है.. खूब हंगामा मचता है कि इस बार जरूर कुछ होगा जरूर कुछ होगा फिर डॉक्टर ऑपरेशन थिएटर से बाहर निकलता है अपना मास्क निकालता है ग्लब्स उतारता है और कहता है सॉरी बस पेट में गैस था.. रिलीज हो गया....!🤣"

# Tokenize the input text
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=MAX_LEN,  # Define MAX_LEN according to your model's requirements
    padding="max_length",  # Pad to a specific length
    truncation=True,  # Explicitly truncate if the text exceeds MAX_LEN
    return_token_type_ids=True,
    return_tensors="pt"  # Return PyTorch tensors
)

# Extract input IDs, attention mask, and token type IDs
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
token_type_ids = inputs["token_type_ids"]

# Perform inference on the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

# Process the outputs as needed for your task (e.g., softmax for probabilities)
# For example, if you're performing classification, you might do:
softmax_outputs = torch.nn.functional.softmax(outputs, dim=1)
predicted_class = torch.argmax(softmax_outputs, dim=1)

# Extract the predicted class value as a Python integer
predicted_class_value = predicted_class.item()

# Map the value of predicted_class_value to the class_names
class_names = [
    'hate,offensive',
    'non-hostile',
    'defamation,offensive',
    'fake',
    'hate',
    'offensive',
    'fake,hate',
    'defamation',
    'defamation,hate',
    'defamation,hate,offensive',
    'defamation,fake,offensive',
    'fake,offensive',
    'defamation,fake',
    'defamation,fake,hate',
    'fake,hate,offensive',
    'defamation,fake,hate,offensive'
]

# Get the corresponding name for the predicted class value
predicted_class_name = class_names[predicted_class_value]
print(predicted_class_name)


offensive


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = MuRILModel(n_classes=16)  # Replace NUM_CLASSES with your actual number of classes
model.load_state_dict(torch.load(PATH))
model.eval()

# Input text
input_text = "@Rofl_RavishNDTV: राहुल गांधी ने प्रधानमंत्री चुनाव से पहले बयान दिया कांग्रेस मुसलमानों की पार्टी है फिर मप्र मे दलित किसान पर भाजपा…"

# Tokenize the input text
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=MAX_LEN,  # Define MAX_LEN according to your model's requirements
    padding="max_length",  # Pad to a specific length
    truncation=True,  # Explicitly truncate if the text exceeds MAX_LEN
    return_token_type_ids=True,
    return_tensors="pt"  # Return PyTorch tensors
)

# Extract input IDs, attention mask, and token type IDs
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
token_type_ids = inputs["token_type_ids"]

# Perform inference on the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

# Process the outputs as needed for your task (e.g., softmax for probabilities)
# For example, if you're performing classification, you might do:
softmax_outputs = torch.nn.functional.softmax(outputs, dim=1)
predicted_class = torch.argmax(softmax_outputs, dim=1)

# Extract the predicted class value as a Python integer
predicted_class_value = predicted_class.item()

# Map the value of predicted_class_value to the class_names
class_names = [
    'hate,offensive',
    'non-hostile',
    'defamation,offensive',
    'fake',
    'hate',
    'offensive',
    'fake,hate',
    'defamation',
    'defamation,hate',
    'defamation,hate,offensive',
    'defamation,fake,offensive',
    'fake,offensive',
    'defamation,fake',
    'defamation,fake,hate',
    'fake,hate,offensive',
    'defamation,fake,hate,offensive'
]

# Get the corresponding name for the predicted class value
predicted_class_name = class_names[predicted_class_value]
print(predicted_class_name)

offensive


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = MuRILModel(n_classes=16)  # Replace NUM_CLASSES with your actual number of classes
model.load_state_dict(torch.load(PATH))
model.eval()

# Input text
input_text = "भारतीय जनता पार्टी (भाजपा) के कार्यकर्ताओं द्वारा सिख समुदाय के एक व्यक्ति को पीटा गया है ।"

# Tokenize the input text
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=MAX_LEN,  # Define MAX_LEN according to your model's requirements
    padding="max_length",  # Pad to a specific length
    truncation=True,  # Explicitly truncate if the text exceeds MAX_LEN
    return_token_type_ids=True,
    return_tensors="pt"  # Return PyTorch tensors
)

# Extract input IDs, attention mask, and token type IDs
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
token_type_ids = inputs["token_type_ids"]

# Perform inference on the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

# Process the outputs as needed for your task (e.g., softmax for probabilities)
# For example, if you're performing classification, you might do:
softmax_outputs = torch.nn.functional.softmax(outputs, dim=1)
predicted_class = torch.argmax(softmax_outputs, dim=1)

# Extract the predicted class value as a Python integer
predicted_class_value = predicted_class.item()

# Map the value of predicted_class_value to the class_names
class_names = [
    'hate,offensive',
    'non-hostile',
    'defamation,offensive',
    'fake',
    'hate',
    'offensive',
    'fake,hate',
    'defamation',
    'defamation,hate',
    'defamation,hate,offensive',
    'defamation,fake,offensive',
    'fake,offensive',
    'defamation,fake',
    'defamation,fake,hate',
    'fake,hate,offensive',
    'defamation,fake,hate,offensive'
]

# Get the corresponding name for the predicted class value
predicted_class_name = class_names[predicted_class_value]
print(predicted_class_name)

fake
